In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cpu device


In [3]:
# Every nn.Module subclass implements the operations on input data in the forward method.
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )
        
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
        
    

In [4]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [10]:
X = torch.rand(1,28,28, device=device)
logits = model(X)
pred_prob = nn.Softmax(dim=1)(logits)
y_pred = pred_prob.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([7])


In [11]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


In [12]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


In [13]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


In [14]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[ 0.2068,  0.1245,  0.2245, -0.1019, -0.1740, -0.0752,  0.5011,  0.8532,
         -0.2201,  0.0484,  0.5526,  0.2189, -0.0933, -0.5377,  0.5373,  0.2395,
          0.4925,  0.1943,  0.0580, -0.0128],
        [ 0.0838,  0.5145,  0.4393, -0.1005, -0.1170, -0.0891,  0.4374,  0.4132,
         -0.3424,  0.2374,  0.3959, -0.0219, -0.0438, -0.5122,  0.4342,  0.2401,
          0.5084, -0.0153,  0.2750, -0.0817],
        [ 0.1221,  0.4598,  0.4241, -0.0226, -0.0726,  0.1750,  0.1989,  0.7867,
         -0.7859,  0.0363,  0.0357,  0.1641, -0.1087, -0.4167,  0.3922,  0.2796,
          0.2504,  0.0261,  0.0104, -0.0732]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.2068, 0.1245, 0.2245, 0.0000, 0.0000, 0.0000, 0.5011, 0.8532, 0.0000,
         0.0484, 0.5526, 0.2189, 0.0000, 0.0000, 0.5373, 0.2395, 0.4925, 0.1943,
         0.0580, 0.0000],
        [0.0838, 0.5145, 0.4393, 0.0000, 0.0000, 0.0000, 0.4374, 0.4132, 0.0000,
         0.2374, 0.3959, 0.0000, 0.0000, 0.0000, 0.43

In [15]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

In [16]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

In [17]:
pred_probab

tensor([[0.0854, 0.1455, 0.1176, 0.0971, 0.1232, 0.1008, 0.0980, 0.0707, 0.0991,
         0.0624],
        [0.0911, 0.1366, 0.1204, 0.0860, 0.1136, 0.1066, 0.1033, 0.0716, 0.1059,
         0.0649],
        [0.0829, 0.1379, 0.1303, 0.0903, 0.1129, 0.1003, 0.0961, 0.0719, 0.1055,
         0.0719]], grad_fn=<SoftmaxBackward0>)

In [18]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[ 0.0304,  0.0148, -0.0109,  ..., -0.0137,  0.0131, -0.0348],
        [-0.0297, -0.0087,  0.0088,  ..., -0.0243,  0.0102, -0.0170]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([-0.0170,  0.0193], grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[ 0.0014, -0.0078, -0.0422,  ...,  0.0312,  0.0311, -0.0207],
        [-0.0361, -0.0247,  0.0191,  ...,  0.0115,  0.0345,  0.0036]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.bias | 